# Trade Studies: DOE + Batch + Pareto Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jman4162/opensatcom/blob/main/notebooks/05_trade_studies.ipynb)

Sweep design parameters, evaluate link budgets, and find the Pareto-optimal designs.

In [ ]:
# Install opensatcom (uncomment for Colab)
# !pip install -q opensatcom

In [ ]:
import numpy as np
import pandas as pd
from opensatcom.trades import (
    RequirementsTemplate,
    DesignOfExperiments,
    BatchRunner,
    extract_pareto_front,
)

## 1. Define Parameter Space

In [ ]:
req = RequirementsTemplate()
req.add("freq_hz", 10e9, 30e9)
req.add("tx_power_w", 20.0, 200.0)
req.add("tx_gain_dbi", 20.0, 45.0)

print(f"Parameters: {req.names}")
print(f"Space: {req.to_parameter_space()}")

## 2. Generate DOE Cases (Latin Hypercube)

In [ ]:
doe = DesignOfExperiments(req.to_parameter_space())
cases = doe.generate(n_samples=100, method="lhs", seed=42)
print(f"Generated {len(cases)} LHS cases")
cases.head()

## 3. Batch Evaluate

In [ ]:
runner = BatchRunner()
results = runner.run(cases)
print(f"Evaluated {len(results)} cases")
results.describe()

## 4. Extract Pareto Front

In [ ]:
# Add a simple cost proxy: power * frequency (higher = more expensive)
results["cost_proxy"] = results["tx_power_w"] * (results["freq_hz"] / 1e9)

# Pareto: minimize cost, maximize margin
pareto_df = extract_pareto_front(
    results, "cost_proxy", "margin_db",
    minimize_x=True, minimize_y=False
)
print(f"Pareto-optimal designs: {len(pareto_df)} / {len(results)}")

## 5. Interactive Pareto Visualization

In [ ]:
from opensatcom.viz.trades import plot_pareto_interactive

fig = plot_pareto_interactive(results, "cost_proxy", "margin_db", pareto_df)
fig.show()

## 6. Parallel Coordinates

In [ ]:
from opensatcom.viz.trades import plot_doe_parallel_coords

fig = plot_doe_parallel_coords(results, objectives=["margin_db"])
fig.show()

---

**Congratulations!** You've completed the OpenSatCom tutorial series.

For CLI usage: `opensatcom --help`